<img src="images/dask_horizontal.svg" align="right" width="30%">

# Table of Contents
* [Bag: Parallel Lists for semi-structured data](#Bag:-Parallel-Lists-for-semi-structured-data)
	* [Creation](#Creation)
	* [Manipulation](#Manipulation)
		* [Exercise: Accounts JSON data](#Exercise:-Accounts-JSON-data)
		* [Basic Queries](#Basic-Queries)
		* [Use `concat` to de-nest](#Use-concat-to-de-nest)
		* [Groupby and Foldby](#Groupby-and-Foldby)
		* [`groupby`](#groupby)
		* [`foldby`](#foldby)
		* [Example with account data](#Example-with-account-data)
		* [Exercise: compute total amount per name](#Exercise:-compute-total-amount-per-name)
	* [DataFrames](#DataFrames)
	* [Limitations](#Limitations)


In [1]:
import distributed
client = distributed.Client()

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

*  [Bag Documenation](http://dask.pydata.org/en/latest/bag.html)
*  [Bag API](http://dask.pydata.org/en/latest/bag-api.html)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc..

In [2]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [8]:
# each element is a text file of JSON lines
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))

49892

In [ ]:
# Requires `s3fs` library
# each element is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv')

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

As with `Array` and `DataFrame` objects, operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution.  

Dask.bag uses `dask.multiprocessing.get` by default.  For examples using dask.bag with a distributed cluster see 

*  [Dask.distributed docs](http://distributed.readthedocs.io/en/latest/)
*  [Blogpost analyzing github data on EC2](https://www.continuum.io/content/dask-distributed-and-anaconda-cluster)

In [9]:
def iseven(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(iseven).map(lambda x: x ** 2)
c

dask.bag<map-lam..., npartitions=10>

In [10]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Exercise: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [12]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "transactions": [{"amount": 1196, "transaction-id": 2587}, {"amount": 979, "transaction-id": 3233}, {"amount": 923, "transaction-id": 3376}, {"amount": 969, "transaction-id": 4338}, {"amount": 975, "transaction-id": 4427}, {"amount": 1119, "transaction-id": 6371}, {"amount": 1082, "transaction-id": 8451}, {"amount": 967, "transaction-id": 9262}, {"amount": 934, "transaction-id": 11592}, {"amount": 1033, "transaction-id": 11729}, {"amount": 861, "transaction-id": 12073}, {"amount": 845, "transaction-id": 12133}, {"amount": 1038, "transaction-id": 12635}, {"amount": 927, "transaction-id": 13702}, {"amount": 986, "transaction-id": 15337}, {"amount": 986, "transaction-id": 15729}, {"amount": 932, "transaction-id": 16268}, {"amount": 1013, "transaction-id": 18242}, {"amount": 943, "transaction-id": 18359}, {"amount": 1025, "transaction-id": 18964}, {"amount": 1013, "transaction-id": 20312}, {"amount": 1050, "transaction-id": 20631}, {"amount": 980, "transaction-id": 21290}, {"am

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [13]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'George',
  'transactions': [{'amount': 1196, 'transaction-id': 2587},
   {'amount': 979, 'transaction-id': 3233},
   {'amount': 923, 'transaction-id': 3376},
   {'amount': 969, 'transaction-id': 4338},
   {'amount': 975, 'transaction-id': 4427},
   {'amount': 1119, 'transaction-id': 6371},
   {'amount': 1082, 'transaction-id': 8451},
   {'amount': 967, 'transaction-id': 9262},
   {'amount': 934, 'transaction-id': 11592},
   {'amount': 1033, 'transaction-id': 11729},
   {'amount': 861, 'transaction-id': 12073},
   {'amount': 845, 'transaction-id': 12133},
   {'amount': 1038, 'transaction-id': 12635},
   {'amount': 927, 'transaction-id': 13702},
   {'amount': 986, 'transaction-id': 15337},
   {'amount': 986, 'transaction-id': 15729},
   {'amount': 932, 'transaction-id': 16268},
   {'amount': 1013, 'transaction-id': 18242},
   {'amount': 943, 'transaction-id': 18359},
   {'amount': 1025, 'transaction-id': 18964},
   {'amount': 1013, 'transaction-id': 20312},
   {'amo

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [14]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 7,
  'name': 'Alice',
  'transactions': [{'amount': -151, 'transaction-id': 21},
   {'amount': -237, 'transaction-id': 520},
   {'amount': -189, 'transaction-id': 944},
   {'amount': -113, 'transaction-id': 1038},
   {'amount': -243, 'transaction-id': 2059},
   {'amount': -154, 'transaction-id': 2653},
   {'amount': -182, 'transaction-id': 2662},
   {'amount': -397, 'transaction-id': 2790},
   {'amount': -224, 'transaction-id': 2879},
   {'amount': -266, 'transaction-id': 3053},
   {'amount': -228, 'transaction-id': 3344},
   {'amount': -167, 'transaction-id': 4211},
   {'amount': -180, 'transaction-id': 4517},
   {'amount': -326, 'transaction-id': 4757},
   {'amount': -247, 'transaction-id': 5147},
   {'amount': -166, 'transaction-id': 5724},
   {'amount': -65, 'transaction-id': 5731},
   {'amount': -381, 'transaction-id': 5809},
   {'amount': -271, 'transaction-id': 6212},
   {'amount': -183, 'transaction-id': 6229},
   {'amount': -286, 'transaction-id': 6255},
   {'amount': 

In [15]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'count': 426, 'name': 'Alice'},
 {'count': 269, 'name': 'Alice'},
 {'count': 135, 'name': 'Alice'},
 {'count': 32, 'name': 'Alice'},
 {'count': 82, 'name': 'Alice'})

In [16]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(426, 269, 135, 32, 82)

In [17]:
# Average number of transactions for all of the Alice's
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

161.74153846153845

### Use `concat` to de-nest

In the example below we see the use of `concat` to flatten results.  We compute the average amount for all transactions for all Alices.

In [18]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'amount': -151, 'transaction-id': 21},
  {'amount': -237, 'transaction-id': 520},
  {'amount': -189, 'transaction-id': 944},
  {'amount': -113, 'transaction-id': 1038},
  {'amount': -243, 'transaction-id': 2059},
  {'amount': -154, 'transaction-id': 2653},
  {'amount': -182, 'transaction-id': 2662},
  {'amount': -397, 'transaction-id': 2790},
  {'amount': -224, 'transaction-id': 2879},
  {'amount': -266, 'transaction-id': 3053},
  {'amount': -228, 'transaction-id': 3344},
  {'amount': -167, 'transaction-id': 4211},
  {'amount': -180, 'transaction-id': 4517},
  {'amount': -326, 'transaction-id': 4757},
  {'amount': -247, 'transaction-id': 5147},
  {'amount': -166, 'transaction-id': 5724},
  {'amount': -65, 'transaction-id': 5731},
  {'amount': -381, 'transaction-id': 5809},
  {'amount': -271, 'transaction-id': 6212},
  {'amount': -183, 'transaction-id': 6229},
  {'amount': -286, 'transaction-id': 6255},
  {'amount': -138, 'transaction-id': 6369},
  {'amount': -123, 'transaction-id': 

In [19]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .concat()
   .take(3))

({'amount': -151, 'transaction-id': 21},
 {'amount': -237, 'transaction-id': 520},
 {'amount': -189, 'transaction-id': 944})

In [20]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .concat()
   .pluck('amount')
   .take(3))

(-151, -237, -189)

In [21]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .concat()
   .pluck('amount')
   .mean()
   .compute())

885.9946733630103

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [22]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [23]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(1, [1, 3, 5, 7, 9]), (0, [0, 2, 4, 6, 8])]

In [24]:
b.groupby(lambda x: x % 2).map(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [25]:
b.foldby(lambda x: x % 2, binop=lambda acc, x: acc if acc > x else x,
                          combine=lambda acc1, acc2: acc1 if acc1 > acc2 else acc2).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [38]:
%%time

# Warning, this one takes a while...
print(js.groupby(lambda item: item['name']).map(lambda k, v: (k, len(v))).compute())

[('Frank', 1000), ('Victor', 750), ('Dan', 850), ('Jerry', 839), ('Yvonne', 700), ('Michael', 650), ('Edith', 800), ('Kevin', 1200), ('Ray', 1400), ('Tim', 1049), ('Zelda', 950), ('Alice', 650), ('Laura', 950), ('Sarah', 1000), ('Patricia', 1150), ('Charlie', 799), ('Quinn', 950), ('Hannah', 1250), ('Ingrid', 992), ('Oliver', 1050), ('Xavier', 950), ('Norbert', 600), ('Ursula', 1250), ('Bob', 1050), ('Wendy', 950), ('George', 1192)]
CPU times: user 1.34 s, sys: 308 ms, total: 1.64 s
Wall time: 5.9 s


In [35]:
%%time

# This one is comparatively fast and produces the same result.
js.foldby(key='name', binop=lambda total, x: total + 1, 
                      initial=0, 
                      combine=lambda a, b: a + b, 
                      combine_initial=0).compute()

CPU times: user 109 ms, sys: 28 ms, total: 137 ms
Wall time: 1.86 s


[('Norbert', 600),
 ('Patricia', 1150),
 ('Hannah', 1250),
 ('Tim', 1049),
 ('Oliver', 1050),
 ('Charlie', 799),
 ('Dan', 850),
 ('Ray', 1400),
 ('Jerry', 839),
 ('Victor', 750),
 ('Ursula', 1250),
 ('Xavier', 950),
 ('Wendy', 950),
 ('Bob', 1050),
 ('Laura', 950),
 ('Sarah', 1000),
 ('Edith', 800),
 ('Quinn', 950),
 ('Ingrid', 992),
 ('Zelda', 950),
 ('Frank', 1000),
 ('Yvonne', 700),
 ('Michael', 650),
 ('Kevin', 1200),
 ('George', 1192),
 ('Alice', 650)]

### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

## DataFrames

Pandas is faster than Pure Python.  In the same way `dask.dataframe` can be faster than `dask.bag`.  We will work more with dataframes later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion - once the data can be made into a data-frame, then complex split-spply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [39]:
df = js.to_dataframe()
df.head()

,id,name,transactions
0,0,George,"[{'amount': 1196, 'transaction-id': 2587}, {'a..."
1,1,Tim,"[{'amount': 61, 'transaction-id': 49}, {'amoun..."
2,2,Kevin,"[{'amount': 2521, 'transaction-id': 151}, {'am..."
3,3,Ingrid,"[{'amount': 1865, 'transaction-id': 84}, {'amo..."
4,4,Frank,"[{'amount': 1753, 'transaction-id': 280}, {'am..."


This is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..  

In [40]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 'name': record['name'], 'amount': transaction['amount'], 'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).concat()
transactions.take(3)

({'amount': 1196, 'id': 0, 'name': 'George', 'transaction-id': 2587},
 {'amount': 979, 'id': 0, 'name': 'George', 'transaction-id': 3233},
 {'amount': 923, 'id': 0, 'name': 'George', 'transaction-id': 3376})

In [41]:
df = transactions.to_dataframe()
df.head()

,amount,id,name,transaction-id
0,1196,0,George,2587
1,979,0,George,3233
2,923,0,George,3376
3,969,0,George,4338
4,975,0,George,4427


This now looks like a well-defined data-frame, and we can apply Pandas-like computations to it efficiently.

In [42]:
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

name
Alice       105132
Bob         369937
Charlie     115019
Dan         195685
Edith       186362
Frank       195123
George      204071
Hannah      303572
Ingrid      225748
Jerry       184433
Kevin       249927
Laura       151326
Michael      61650
Norbert      72565
Oliver       88806
Patricia    199387
Quinn       215482
Ray         288505
Sarah       223443
Tim         189690
Ursula      319255
Victor      159584
Wendy       144931
Xavier      187602
Yvonne      178688
Zelda       184077
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.